In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
import requests

In [66]:
arrest_data = requests.get('https://www.ethnicity-facts-figures.service.gov.uk/crime-justice-and-the-law/policing/number-of-arrests/latest/downloads/number-of-arrests.csv').content
arrest_df = pd.read_csv(io.StringIO(arrest_data.decode('utf-8')))
arrest_df = arrest_df.drop(columns=['Notes'])
arrest_df = arrest_df.dropna()
arrest_df = arrest_df[arrest_df['Age_Group'] == 'All']
arrest_df = arrest_df[arrest_df['Gender'] == 'All']
arrest_df['id'] = arrest_df.index

In [20]:
pop_df = pd.read_excel('/content/ukmidyearestimates20192020ladcodes.xls', sheet_name='MYE 5', header=4)
pop_df = pop_df.dropna()

In [65]:
arrest_df

,Measure,Time,Time_type,Ethnicity,Ethnicity_type,Gender,Gender_type,Age_Group,Age_Group_type,Geography,Geography_type,Number of arrests,"Population by ethnicity, gender, and PFA (based on 2011 Census)","Rate per 1,000 population by ethnicity, gender, and PFA","Proportion of arrests of this ethnicity, in this year, of this gender and in this police force area (excludes unreported)",id
53352,Number of arrests,2006/07,Financial year,Asian,ONS 2001 5+1,All,Male/Female,All,Arrest age group categories,Avon and Somerset,Police Force Area,489,"33,005",15,1.50%,53352
53353,Number of arrests,2006/07,Financial year,Asian,ONS 2001 5+1,All,Male/Female,All,Arrest age group categories,Bedfordshire,Police Force Area,"1,844","81,480",23,11.90%,53353
53354,Number of arrests,2006/07,Financial year,Asian,ONS 2001 5+1,All,Male/Female,All,Arrest age group categories,Cambridgeshire,Police Force Area,796,"39,535",20,4.60%,53354
53355,Number of arrests,2006/07,Financial year,Asian,ONS 2001 5+1,All,Male/Female,All,Arrest age group categories,Cheshire,Police Force Area,177,"12,794",14,0.80%,53355
53356,Number of arrests,2006/07,Financial year,Asian,ONS 2001 5+1,All,Male/Female,All,Arrest age group categories,Cleveland,Police Force Area,405,"17,419",23,1.90%,53356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218587,Number of arrests,2013/14,Financial year,Mixed White and Asian,ONS 2001 16+1,All,Male/Female,All,Arrest age group categories,All,Police Force Area,"2,906","341,727",9,0.30%,218587
218588,Number of arrests,2014/15,Financial year,Mixed White and Asian,ONS 2001 16+1,All,Male/Female,All,Arrest age group categories,All,Police Force Area,"2,711","341,727",8,0.30%,218588
218589,Number of arrests,2015/16,Financial year,Mixed White and Asian,ONS 2001 16+1,All,Male/Female,All,Arrest age group categories,All,Police Force Area,"2,532","341,727",7,0.30%,218589
218590,Number of arrests,2016/17,Financial year,Mixed White and Asian,ONS 2001 16+1,All,Male/Female,All,Arrest age group categories,All,Police Force Area,"2,289","341,727",7,0.30%,218590


In [51]:
pop_df

,Code,Name,Geography1,Area (sq km),Estimated Population mid-2019,2019 people per sq. km,Estimated Population mid-2018,2018 people per sq. km,Estimated Population mid-2017,2017 people per sq. km,Estimated Population mid-2016,2016 people per sq. km,Estimated Population mid-2015,2015 people per sq. km,Estimated Population mid-2014,2014 people per sq. km,Estimated Population mid-2013,2013 people per sq. km,Estimated Population mid-2012,2012 people per sq. km,Estimated Population mid-2011,2011 people per sq. km,Estimated Population mid-2010,2010 people per sq. km,Estimated Population mid-2009,2009 people per sq. km,Estimated Population mid-2008,2008 people per sq. km,Estimated Population mid-2007,2007 people per sq. km,Estimated Population mid-2006,2006 people per sq. km,Estimated Population mid-2005,2005 people per sq. km,Estimated Population mid-2004,2004 people per sq. km,Estimated Population mid-2003,2003 people per sq. km,Estimated Population mid-2002,2002 people per sq. km,Estimated Population mid-2001,2001 people per sq. km
0,K02000001,UNITED KINGDOM,Country,242743.0,66796807.0,275.0,66435550.0,274.0,66040229.0,272.0,65648054.0,270.0,65110034.0,268.0,64596752.0,266.0,64105654.0,264.0,63705030.0,262.0,63285145.0,261.0,62759456.0,259.0,62260486.0,256.0,61823772.0,255.0,61319075.0,253.0,60827067.0,251.0,60413276.0,249.0,59950364.0,247.0,59636662.0,246.0,59365677.0,245.0,59113016.0,244.0
1,K03000001,GREAT BRITAIN,Country,228950.0,64903140.0,283.0,64553909.0,282.0,64169395.0,280.0,63785917.0,279.0,63258413.0,276.0,62756254.0,274.0,62275929.0,272.0,61881396.0,270.0,61470827.0,268.0,60954623.0,266.0,60467153.0,264.0,60044620.0,262.0,59557392.0,260.0,59083954.0,258.0,58685543.0,256.0,58236322.0,254.0,57931738.0,253.0,57668143.0,252.0,57424178.0,251.0
2,K04000001,ENGLAND AND WALES,Country,151047.0,59439840.0,394.0,59115809.0,391.0,58744595.0,389.0,58381217.0,387.0,57885413.0,383.0,57408654.0,380.0,56948229.0,377.0,56567796.0,375.0,56170927.0,372.0,55692423.0,369.0,55235253.0,366.0,54841720.0,363.0,54387392.0,360.0,53950854.0,357.0,53575343.0,355.0,53152022.0,352.0,52863238.0,350.0,52602143.0,348.0,52359978.0,347.0
3,E92000001,ENGLAND,Country,130311.0,56286961.0,432.0,55977178.0,430.0,55619430.0,427.0,55268067.0,424.0,54786327.0,420.0,54316618.0,417.0,53865817.0,413.0,53493729.0,411.0,53107169.0,408.0,52642452.0,404.0,52196381.0,401.0,51815853.0,398.0,51381093.0,394.0,50965186.0,391.0,50606034.0,388.0,50194600.0,385.0,49925517.0,383.0,49679267.0,381.0,49449746.0,379.0
4,E12000001,NORTH EAST,Region,8579.0,2669941.0,311.0,2657909.0,310.0,2644727.0,308.0,2636589.0,307.0,2624579.0,306.0,2618736.0,305.0,2610563.0,304.0,2602387.0,303.0,2596441.0,303.0,2586868.0,302.0,2575441.0,300.0,2569312.0,299.0,2562045.0,299.0,2552568.0,298.0,2547127.0,297.0,2540502.0,296.0,2540461.0,296.0,2540600.0,296.0,2540090.0,296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,N09000006,Fermanagh and Omagh,Local Government District,2864.0,117397.0,41.0,116835.0,41.0,116289.0,41.0,115799.0,40.0,115311.0,40.0,114992.0,40.0,114365.0,40.0,114230.0,40.0,113501.0,40.0,112919.0,39.0,111998.0,39.0,111494.0,39.0,110345.0,39.0,109344.0,38.0,108586.0,38.0,107923.0,38.0,107332.0,37.0,106663.0,37.0,105751.0,37.0
422,N09000007,Lisburn and Castlereagh,Local Government District,506.0,146002.0,289.0,144381.0,285.0,142640.0,282.0,141181.0,279.0,140205.0,277.0,138627.0,274.0,136808.0,270.0,136311.0,270.0,135280.0,267.0,133909.0,265.0,132386.0,262.0,130584.0,258.0,129179.0,255.0,128185.0,253.0,127043.0,251.0,125764.0,249.0,125001.0,247.0,124757.0,247.0,124585.0,246.0
423,N09000008,Mid and East Antrim,Local Government District,1059.0,139274.0,131.0,138773.0,131.0,138152.0,130.0,137821.0,130.0,137145.0,129.0,136642.0,129.0,135997.0,128.0,135838.0,128.0,135365.0,128.0,135157.0,128.0,134774.0,127.0,134169.0,127.0,133270.0,126.0,132021.0,125.0,130980.0,124.0,12

In [67]:
merge_df = pd.merge(left=arrest_df, left_on='Geography', right=pop_df, right_on='Name')
merge_df['id'] = merge_df.index

In [70]:
merge_df.to_csv('output_test.csv')

Arrests in the UK by ethnicity by area across uk (city(urban area), small town/village(rural b area)) 

First use a classification method to classify what sort of area places are

get arrest data on a mix of different places from each category from different regions in the uk over a set period




How can you remove ai bias from an algorithm which is designed to find social bias?


Filter arrest data by all age, all genders
